In [1]:
#Load libraries
pack_lib = '/home/j/temp/dccasey/temperature/packages/'
.libPaths(pack_lib)
library('parallel')
library('sp')
library('rgdal', lib.loc = '/home/j/temp/geospatial/packages')
for(ppp in c('raster','ncdf4','data.table','foreach', 'snow')){
    library(ppp, lib.loc = pack_lib, character.only =T)
}

rgdal: version: 1.2-4, (SVN revision 643)
 Geospatial Data Abstraction Library extensions to R successfully loaded
 Loaded GDAL runtime: GDAL 1.7.3, released 2010/11/10
 Path to GDAL shared files: /usr/share/gdal
 GDAL does not use iconv for recoding strings.
 Loaded PROJ.4 runtime: Rel. 4.7.1, 23 September 2009, [PJ_VERSION: 470]
 Path to PROJ.4 shared files: (autodetected)
 Linking to sp version: 1.2-3 

Attaching package: ‘data.table’

The following object is masked from ‘package:raster’:

    shift


Attaching package: ‘snow’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, clusterSplit, makeCluster, parApply,
    parCapply, parLapply, parRapply, parSapply, splitIndices,
    stopCluster



In [2]:
#set raster options
#slots = 25
#gigs_per_cell = 0.000000007511822 #based off some random tests
#num_of_cells_in_mem = ((25*2)-5)/gigs_per_cell #allow 5 gigs of overhead
#rasterOptions(chunksize = num_of_cells_in_mem/10)
rasterOptions(maxmemory = 1e9)

In [3]:
rasterOptions()

format        : raster 
datatype      : FLT8S 
overwrite     : FALSE 
progress      : none 
timer         : FALSE 
chunksize     : 1e+07 
maxmemory     : 1e+09 
tmpdir        : /tmp/Rtmp78CVB8/raster// 
tmptime       : 168 
setfileext    : TRUE 
tolerance     : 0.1 
standardnames : TRUE 
warn depracat.: TRUE 
header        : none


In [4]:
#set some paths
data.dir = '/home/j/temp/dccasey/temperature/data/cru_ts/'
pop.dir = '/home/j/WORK/01_covariates/02_inputs/population_counts/outputs/full_ts_1980_2015/'

In [5]:
cru_years = c('1981.1990', '1991.2000','2001.2010','2011.2015')
cru_max = lapply(paste0(data.dir,'cru_ts3.24.',cru_years,'.tmx.dat.nc'), function(x) brick(x))
cru_min = lapply(paste0(data.dir,'cru_ts3.24.',cru_years,'.tmn.dat.nc'), function(x) brick(x))

In [6]:
#drop years we don't need
nine = names(cru_max[[1]])[as.numeric(substr(names(cru_max[[1]]),2,5))>=1989]
cru_max[[1]] = cru_max[[1]][[nine]]
cru_min[[1]] = cru_min[[1]][[nine]]

In [7]:
#brick the crus
cru_max = brick(cru_max)
cru_min = brick(cru_min)

In [8]:
#drop the days from the names
names(cru_max) = substr(names(cru_max), 1,nchar(names(cru_max))-3)
names(cru_min) = substr(names(cru_min), 1,nchar(names(cru_min))-3)

In [9]:
dim(cru_max)

[1] 360 720 324

In [10]:
names(cru_max)[dim(cru_max)[3]]

[1] "X2015.12"

In [11]:
#number of days per month
#from Jan 1 1989 - Dec 31 2016 there are 10,226 days
num_day_month = c(31,28,31,30,31,30,31,31,30,31,30,31)
days = as.character(seq(as.Date('1989/01/01'), as.Date('2015/12/31'), by = 'day'))
months = as.character(seq(as.Date('1989/01/01'), as.Date('2015/12/01'), by = 'month'))
months = substr(months, 1,nchar(months)-3)

In [12]:
#Find the mid point
mid_points = as.vector(lapply(months, function(x) grep(x,days,value =T)[round(length(grep(x,days,value =T))/2)]))


In [13]:
#find the mid_point_id
mp_loc = days %in% mid_points
mp_day_locs = (1:length(days))[mp_loc]

In [14]:
interp_row_simple <- function(value_vector) {
  if (any(is.na(value_vector))) {
    return(rep(NA, length(1:length(days))))
  } else {
    return(approx(x = mp_day_locs, y= value_vector, xout = 1:length(days), rule=2)$y)
  }
}

In [15]:
spline_row_simple <- function(value_vector) {
  if (any(is.na(value_vector))) {
    return(rep(NA, length(1:length(days))))
  } else {
    return(spline(x = mp_day_locs, y= value_vector, xout = 1:length(days))$y)
  }
}

In [16]:
#start the cluster
#beginCluster(4, exclude = c('rgdal','raster'))

In [17]:
#max_interp = clusterR(cru_max, calc, args = list(fun = interp_row_simple), export = c('days', 'mp_day_locs'))

In [18]:
#min_interp = clusterR(cru_min, calc, args = list(fun = interp_row_simple), export = c('days', 'mp_day_locs'))

In [19]:
#mean_interp = (max_interp+min_interp)/2

In [20]:
#clean up workspace
#rm(max_interp)
#rm(min_interp)

In [21]:
#end cluster
#endCluster()

In [22]:
#save the mean interp object

In [23]:
#wahoo parallel
#ptm <- proc.time()
#beginCluster(round(slots *7), exclude = c('rgdal','raster'))
#max_interp = clusterR(cru_max, calc, args = list(fun = interp_row_simple), export = c('days', 'mp_day_locs'))
#endCluster()
#runtime = proc.time() - ptm


In [24]:
#max_interp = clusterR(cru_max, calc, args = list(fun = interp_row_simple), export = c('days', 'mp_day_locs'))
#min_interp = clusterR(cru_min, calc, args = list(fun = interp_row_simple), export = c('days', 'mp_day_locs'))
#mean_interp = (max_interp+min_interp)/2

In [25]:
max_interp = calc(cru_max, function(y) spline_row_simple(y))

In [26]:
min_interp = calc(cru_min, function(y) spline_row_simple(y))

In [27]:
mean_interp = (max_interp+min_interp)/2
rm(max_interp)
rm(min_interp)

In [29]:
names(mean_interp) = days

In [30]:
for(nnn in unique(substr(names(mean_interp),1,5))){
    writeRaster(mean_interp[[grep(nnn,names(mean_interp))]], paste0('/share/geospatial/temperature/estimates/cru_spline_interp/',nnn,'_s_mean_interp.ncdf'))
}

In [ ]:
summary(mean_interp[[2]])

In [ ]:
#save(mean_interp, file = '/home/j/temp/dccasey/temperature/mean_cru_interp_2.Rdata')

In [ ]:
#writeRaster(mean_interp, file = '/home/j/temp/dccasey/temperature/mean_cru_interp_2.ncdf')

In [ ]:
#write a raster per year
names(mean_interp)[1:10]

In [ ]:
dim(mean_interp)
length(days)
names(mean_interp)[1]

In [ ]:
summary(mean_interp[[15:18]],na.rm=T)

In [ ]:
plot(mean_interp[[5]])